In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Importing API key
from api_keys import g_key


In [2]:
#Reading and displaying the output file from Part 1
path_file = "../WeatherPy/output_data_file"
data_file = pd.read_csv(path_file)
data_file.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Puerto Ayora,20,EC,1583959155,70,-0.74,-90.35,86.00,9.17
1,Avarua,46,CK,1583959030,74,-21.21,-159.78,84.20,9.17
2,Pevek,99,RU,1583959157,87,69.70,170.31,-9.56,6.33
3,Ambanja,100,MG,1583959158,89,-13.68,48.45,78.87,2.93
4,Tocopilla,0,CL,1583959159,39,-22.09,-70.20,72.01,10.94


In [12]:
#Configuring gmaps

gmaps.configure(api_key=g_key)

coordinates = data_file[["Latitude","Longitude"]]
humidity = data_file["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '950px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plotting Heatmap
fig = gmaps.figure(layout =figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False,max_intensity=humidity.max(),
                                 point_radius=3)


# Adding layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions.

vacation_file = data_file.loc[(data_file["Max Temperature"]<80) & (data_file["Max Temperature"]>70),:]
vacation_file = vacation_file.loc[(vacation_file["Wind Speed"]<10) & (vacation_file["Cloudiness"]==0),:]
vacation_file

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
41,Carnarvon,0,AU,1583959187,100,-24.87,113.63,71.60,3.36
83,Sumenep,0,ID,1583959258,83,-7.02,113.87,79.16,3.76
131,Castro,0,BR,1583959126,63,-24.79,-50.01,72.82,2.82
144,Mount Isa,0,AU,1583959044,65,-20.73,139.50,77.00,5.82
181,Adrar,0,MR,1583959377,7,20.50,-10.07,72.41,7.34
221,Inhambane,0,MZ,1583959426,76,-23.86,35.38,78.10,8.84
250,Kununurra,0,AU,1583959461,94,-15.77,128.73,73.40,0.94
274,Kassala,0,SD,1583959488,46,15.45,36.40,78.12,7.02
316,Porto União,0,BR,1583959542,39,-26.24,-51.08,74.66,2.62
332,Rawson,0,AR,1583959560,56,-43.30,-65.10,70.47,0.27


In [5]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

#Hotel Map
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "restaurant",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
for index, row in vacation_file.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    params["location"] = f"{lat},{lng}"

    # run a request using params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    hotel_data = response.json()
    results = hotel_data["results"]

    # Print the json
    try:
        print(f"The first hotel within 5000m in the City of {city} is {results[0]['name']}.")
        
        vacation_file.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("No Hotels found... skipping.")
        
    print("------------")

The first hotel within 5000m in the City of Carnarvon is Hospitality Carnarvon.
------------
The first hotel within 5000m in the City of Sumenep is Family Nur Hotel.
------------
The first hotel within 5000m in the City of Castro is Sansey Restaurante Oriental.
------------
The first hotel within 5000m in the City of Mount Isa is Subway.
------------
No Hotels found... skipping.
------------
The first hotel within 5000m in the City of Inhambane is África Tropical.
------------
The first hotel within 5000m in the City of Kununurra is The Kimberley Grande Resort.
------------
The first hotel within 5000m in the City of Kassala is مطعم اسمايل Smile.
------------
The first hotel within 5000m in the City of Porto União is Hotel Flórida.
------------
The first hotel within 5000m in the City of Rawson is El Fideo Feliz.
------------


In [7]:
#Drop any rows with null values.
hotel_df = vacation_file.dropna(how="any")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
41,Carnarvon,0,AU,1583959187,100,-24.87,113.63,71.60,3.36,Hospitality Carnarvon
83,Sumenep,0,ID,1583959258,83,-7.02,113.87,79.16,3.76,Family Nur Hotel
131,Castro,0,BR,1583959126,63,-24.79,-50.01,72.82,2.82,Sansey Restaurante Oriental
144,Mount Isa,0,AU,1583959044,65,-20.73,139.50,77.00,5.82,Subway
221,Inhambane,0,MZ,1583959426,76,-23.86,35.38,78.10,8.84,África Tropical
250,Kununurra,0,AU,1583959461,94,-15.77,128.73,73.40,0.94,The Kimberley Grande Resort
274,Kassala,0,SD,1583959488,46,15.45,36.40,78.12,7.02,مطعم اسمايل Smile
316,Porto União,0,BR,1583959542,39,-26.24,-51.08,74.66,2.62,Hotel Flórida
332,Rawson,0,AR,1583959560,56,-43.30,-65.10,70.47,0.27,El Fideo Feliz


In [8]:
#Plot markers on top of the heatmap.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…